## Imports 

In [18]:
import common.feature_string as feature_string
import common.feature_num as feature_num
import common.feature_categorica as feature_categorica
import common.common_machine_learning as common
import common.my_pipeline as my_pipe
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
import xgboost as xgb
import time
from hyperopt import fmin, tpe, hp, STATUS_OK
from sklearn.model_selection import cross_val_score

### Ignore Warnings 

In [2]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

## Constantes

In [3]:
TARGET = 'precio'
K = 3
RANDOM_SEMILLA = 3 # numero favorito (de la suerte)
XGB_RE_OBJECTIVE = 'reg:squarederror'
N_ITER = 15

In [4]:
t0 = time.time()
train = common.cargar_set_optimizado('sets_de_datos/train.csv', index_col = 0)
t1 = time.time()
print('Tiempo = {0:.2f} minutos'.format((t1-t0)/60))

Tiempo = 0.06 minutos


## Features agregables por registro (no necesita entrenamiento previo)

Esto puede tardar unos minutos.  
Dima'pc = 3 minutos (aprox)

In [5]:
t0 = time.time()
feature_num.agregar_feature_fecha_numerica(train)
train = feature_categorica.agregar_feature_one_hot_encoding(train)
feature_string.agregar_feature_todos_ref(train)
t1 = time.time()
print('Tiempo = {0:.2f} minutos'.format((t1-t0)/60))

Tiempo = 7.87 minutos


Elimino features no usar

In [6]:
feature_num.eliminar_num_no_feature(train)
feature_categorica.eliminar_categoria_no_feature(train)
feature_string.eliminar_string_no_feature(train)

## Muestra y Target

In [7]:
X = train.drop([TARGET], axis = 1).copy()
y = train[TARGET].copy()

## Mejor resultado tunneo 2 y 2 bis

Simplificado

## Tunning Overfit

In [13]:
hyper_parametros = {
    'colsample_bytree': 0.76, 
    'gamma': 0, 
    'lambda': 2.77, 
    'learning_rate': 0.0079, 
    'max_depth': 134, 
    'min_child_weight': 1, 
    'n_estimators':  470, 
    'subsample': 0.68
}

xgb_model = XGBRegressor(objective = XGB_RE_OBJECTIVE, **hyper_parametros)

pre_procesar_pipe = my_pipe.get_columns_pipeline()
busqueda_pipe = Pipeline(steps = [
        ('pre-procesar', pre_procesar_pipe),
        ('xgb_regressor', )
    ])  

t0 = time.time()
score = cross_val_score(busqueda_pipe, X, y, scoring='neg_mean_absolute_error', cv=K).mean()
t1 = time.time()
print("SCORE: {:.3f} params {}".format(score, hyper_parametros))
print('Tiempo = {0:.2f} minutos'.format((t1-t0)/60))

SCORE: -923621.167 params {'colsample_bytree': 0.76, 'gamma': 0, 'lambda': 2.77, 'learning_rate': 0.0079, 'max_depth': 134, 'min_child_weight': 1, 'n_estimators': 470, 'subsample': 0.68}
Tiempo = 13.46 minutos


Veamos si los mejores hiperparametros del random forest sirven para XGboost